# Aplicação de Pesquisa Operacional - PPGMNE 2021

## Antonio Carlos da Silva Júnior

### Prof. Gustavo Valentim Loch

In [1]:
from copy import copy
import pandas as pd
import logging

logging.basicConfig(level = logging.INFO)
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Preparação dos dados de entrada

In [2]:
S = {'h': 10, 'w': 16}

beta = 0.1

b = [2,2,2,2]

pi_max = beta*S['h']*S['w']

R = {}
R[0] = {'h': 3, 'w': 8}
R[1] = {'h': 7, 'w': 5}
R[2] = {'h': 3, 'w': 6}
R[3] = {'h': 4, 'w': 6}

# Funções

In [3]:
def __validar_T(T, pi_max, b, S):
    v1 = sum([T['g'][x] <= b[x] for x in T['g']]) > 0 # verifica respeita b
    v2 = T['d']['p'] <= pi_max # verifica se respeita beta
    v3 = T['d']['h'] <= S['h'] and T['d']['w'] <= S['w'] # verifica se respeita S
    return v1 and v2 and v3
    
def obter_T(h0, h1, w0, w1, g0, g1, p0, p1, o, pi_max, b, S):
    T = {}
    T['d'] = {}
    if o == 'h':
        T['d']['h'] = max(h0, h1)
        T['d']['w'] = w0 + w1
    else:
        T['d']['h'] = h0 + h1
        T['d']['w'] = max(w0, w1)
        
    T['d']['p'] = T['d']['h']*T['d']['w'] - h0*w0 - h1*w1 + p0 + p1
    T['g'] = {k: g0.get(k, 0) + g1.get(k, 0) for k in set(g0)}
    
    return T, __validar_T(T, pi_max, b, S)

# Início do algoritmo

In [4]:
L = [{}]
for i in R:
    L[0][i] = {}
    L[0][i]['d'] = {}
    L[0][i]['d']['h'] = R[i]['h']
    L[0][i]['d']['w'] = R[i]['w']
    L[0][i]['d']['p'] = 0
    L[0][i]['g'] = {}
    for j in R:
        if i == j:
            L[0][i]['g'][j] = 1
        else:
            L[0][i]['g'][j] = 0
    L[0][i]['c'] = (i, None, None)

In [5]:
F = copy(L)
k = 0

In [6]:
while len(F[k]) > 0:
    logging.info(f'Iteração {k}')
    logging.info(f'Combinações acumuladas: {len(L[k])}')
    k += 1
    
    C = [(i,j,o) for i in F[k-1].keys() for j in F[k-1].keys() for o in ['h', 'v']]

    if k > 1:
        C += [(i,j,o) for i in F[k-1].keys() for j in F[k-2].keys() for o in ['h', 'v']]
        
    idx = max(list(L[k-1].keys()))
    F.append({})
    
    for c in C:
        r0 = L[k-1][c[0]]
        r1 = L[k-1][c[1]]
        T, valido = obter_T(
            h0=r0['d']['h'], 
            h1=r1['d']['h'], 
            w0=r0['d']['w'], 
            w1=r1['d']['w'], 
            p0=r0['d']['p'],
            p1=r1['d']['p'],
            g0=r0['g'], 
            g1=r1['g'],
            o=c[2], 
            pi_max=pi_max, 
            b=b, 
            S=S)
        if valido:
            lst_f = [{'d':F[k][s]['d'], 'g':F[k][s]['g']} for s in F[k]] if len(F[k]) > 0 else []
            lst_l = [{'d':L[k-1][r]['d'], 'g': L[k-1][r]['g']} for r in L[k-1]]
            if {'d':T['d'], 'g':T['g']} not in lst_l + lst_f:
                T['c'] = c
                idx += 1
                F[k][idx] = T
                
    L.append({**L[k-1], **F[k]})
dict_out = L[len(L)-1]
logging.info(f'Fim!')

INFO:root:Iteração 0
INFO:root:Combinações acumuladas: 4
INFO:root:Iteração 1
INFO:root:Combinações acumuladas: 18
INFO:root:Iteração 2
INFO:root:Combinações acumuladas: 60
INFO:root:Iteração 3
INFO:root:Combinações acumuladas: 98
INFO:root:Fim!


# Dados de saída:

In [7]:
df_out = None
for key in dict_out:
    df_dim = pd.DataFrame([dict_out[key]['d']])
    df_comb = pd.DataFrame([dict_out[key]['c']])
    df_comb.columns = ['r1', 'r2', 'o']
    df = pd.concat([df_dim, df_comb], axis=1)
    
    if df_out is None:
        df_out = df.copy()
    else:
        df_out = pd.concat([df_out, df], ignore_index=True)

### Legenda:

* h: altura da combinação
* w: comprimento da combinação
* p: perda
* r1: peça de origem 1
* r2: peça de origem 2
* o: orientação da combinação das peças de origem

In [8]:
df_out

,h,w,p,r1,r2,o
0,3,8,0,0,None,None
1,7,5,0,1,None,None
2,3,6,0,2,None,None
3,4,6,0,3,None,None
4,3,16,0,0,0,h
5,6,8,0,0,0,v
6,3,14,0,0,2,h
7,6,8,6,0,2,v
8,4,14,8,0,3,h
9,7,8,8,0,3,v
